# The purpose of this script is to complete the datas in clean_data (from hossain)

## Libraries to install

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/memoire'

/content/drive/MyDrive/memoire


In [ ]:
! pip install datasets
! pip install transformers

In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd

## MNLI

### clean_data (from hossain)

In [ ]:
data_files = 'data/new_benchmarks/processed_for_run/MNLI/dev_matched.tsv'
dataset_mnli = load_dataset('csv', data_files=data_files, sep='\t')

Using custom data configuration default-5af8b8d0034dcbbe
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-5af8b8d0034dcbbe/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_mnli

DatasetDict({
    train: Dataset({
        features: ['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'label2', 'label3', 'label4', 'label5', 'gold_label'],
        num_rows: 1500
    })
})

In [ ]:
text_mnli = dataset_mnli['train']['sentence1']
hypothesis_mnli = dataset_mnli['train']['sentence2']

In [ ]:
label_mnli = dataset_mnli['train']['gold_label']

In [ ]:
text_pos_id = np.arange(1, 1500, 3)

text_mnli_pos = []
hypothesis_mnli_pos = []
text_mnli_neg = []
hypothesis_mnli_neg = []
for i in text_pos_id:
  text_mnli_pos.append(text_mnli[i])
  hypothesis_mnli_pos.append(hypothesis_mnli[i-1])
  text_mnli_neg.append(text_mnli[i-1])
  hypothesis_mnli_neg.append(hypothesis_mnli[i])

### original data 

In [ ]:
data_files = ['corpus/MNLI/original/multinli_1.0_train.jsonl', 'corpus/MNLI/original/multinli_1.0_dev_mismatched.jsonl', 'corpus/MNLI/original/multinli_1.0_dev_matched.jsonl' ]
dataset_mnli_origninal = load_dataset('json', data_files=data_files)

Using custom data configuration default-4ecce22f36f15b61


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4ecce22f36f15b61/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
text_mnli_original = dataset_mnli_origninal["train"]['sentence1']
hypothesis_mnli_original = dataset_mnli_origninal["train"]['sentence2']

In [ ]:
label_mnli_original = dataset_mnli_origninal["train"]["gold_label"]

In [ ]:
hypothesis_mnli_original_strip = []
for x in hypothesis_mnli_original:
    hypothesis_mnli_original_strip.append(x.strip())

In [ ]:
list_res_mnli = {}
list_error_mnli = []
for i, h in enumerate(hypothesis_mnli_pos):
  h = h.strip()
  if h in hypothesis_mnli_original_strip:
    index = hypothesis_mnli_original_strip.index(h)
    label = label_mnli_original[index]
    list_res_mnli[i] = label
  else:
    list_error_mnli.append((i,h))

In [ ]:
print(len(list_res_mnli))

495


In [ ]:
print(list_error_mnli)

[80, 257, 267, 328, 455]


In [ ]:
# 手动添加有错误的label
list_res_mnli[80]='neutral'
list_res_mnli[257]='entailment'
list_res_mnli[267]='entailment'
list_res_mnli[328]='contradiction'
list_res_mnli[455]='entailment'

In [ ]:
# 检查一下
print(len(list_res_mnli))

500


In [ ]:
list_final_mnli=[]
for i in range(len(list_res_mnli)):
    list_final_mnli.append([text_mnli_pos[i],hypothesis_mnli_pos[i],list_res_mnli[i]])
    list_final_mnli.append([text_mnli_neg[i],hypothesis_mnli_pos[i], label_mnli[i*3+0]])
    list_final_mnli.append([text_mnli_pos[i],hypothesis_mnli_neg[i], label_mnli[i*3+1]])
    list_final_mnli.append([text_mnli_neg[i],hypothesis_mnli_neg[i], label_mnli[i*3+2]])
 
column_name = ['text', 'hypothesis', 'label']
csv_name='./data/mnli_completed2.tsv'
xml_df = pd.DataFrame(list_final_mnli, columns=column_name)
xml_df.to_csv(csv_name, sep='\t')

## SNLI

### clean data

In [ ]:
data_files = 'data/new_benchmarks/processed_for_run/SNLI/dev.tsv'
dataset_snli = load_dataset('csv', data_files=data_files, sep='\t')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-787e7e1dd2848702/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_snli

DatasetDict({
    train: Dataset({
        features: ['index', 'captionID', 'pairID', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'gold_label'],
        num_rows: 1500
    })
})

In [ ]:
text_snli = dataset_snli['train']['sentence1']
hypothesis_snli = dataset_snli['train']['sentence2']

In [ ]:
label_snli = dataset_snli['train']['gold_label']

In [ ]:
text_pos_id = np.arange(1, 1500, 3)

text_snli_pos = []
hypothesis_snli_pos = []
text_snli_neg = []
hypothesis_snli_neg = []
for i in text_pos_id:
  text_snli_pos.append(text_snli[i])
  hypothesis_snli_pos.append(hypothesis_snli[i-1])
  text_snli_neg.append(text_snli[i-1])
  hypothesis_snli_neg.append(hypothesis_snli[i])

### original data 

In [ ]:
data_files = ['corpus/SNLI/original/snli_1.0_dev.jsonl', 'corpus/SNLI/original/snli_1.0_train.jsonl']
dataset_snli_origninal = load_dataset('json', data_files=data_files)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_snli_origninal

DatasetDict({
    train: Dataset({
        features: ['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse'],
        num_rows: 560152
    })
})

In [ ]:
text_snli_original = dataset_snli_origninal["train"]['sentence1']
hypothesis_snli_original = dataset_snli_origninal["train"]['sentence2']

In [ ]:
label_snli_original = dataset_snli_origninal["train"]["gold_label"]

In [ ]:
hypothesis_snli_original_strip = []
for x in hypothesis_snli_original:
    hypothesis_snli_original_strip.append(x.strip())

In [ ]:
list_res_snli = {}
list_error_snli = []
for i, h in enumerate(hypothesis_snli_pos):
  h = h.strip()
  if h in hypothesis_snli_original_strip:
    index = hypothesis_snli_original_strip.index(h)
    label = label_snli_original[index]
    list_res_snli[i] = label
  else:
    list_error_snli.append((i,h))

In [ ]:
print(len(list_res_snli))
# list_res_snli

In [ ]:
print(len(list_error_snli))
list_error_snli

2


[(370, 'The person is sleeping in the alley way.'), (375, 'Women are puking')]

In [ ]:
list_res_snli[370]='contradiction'
list_res_snli[375]='neutral'

In [ ]:
print(len(list_res_snli))

500


In [ ]:
list_final_snli=[]
for i in range(len(list_res_snli)):
    list_final_snli.append([text_snli_pos[i],hypothesis_snli_pos[i],list_res_snli[i]])
    list_final_snli.append([text_snli_neg[i],hypothesis_snli_pos[i], label_snli[i*3+0]])
    list_final_snli.append([text_snli_pos[i],hypothesis_snli_neg[i], label_snli[i*3+1]])
    list_final_snli.append([text_snli_neg[i],hypothesis_snli_neg[i], label_snli[i*3+2]])
 
column_name = ['text', 'hypothesis', 'label']
csv_name='./data/snli_completed.tsv'
xml_df = pd.DataFrame(list_final_snli, columns=column_name)
xml_df.to_csv(csv_name, sep='\t')

## RTE

### clean data

In [ ]:
data_files = 'data/new_benchmarks/processed_for_run/RTE/dev.tsv'
dataset_rte = load_dataset('csv', data_files=data_files, sep='\t')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-16efd731ce8e74d7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_rte

DatasetDict({
    train: Dataset({
        features: ['index', 'sentence1', 'sentence2', 'gold_label'],
        num_rows: 1500
    })
})

In [ ]:
text_rte = dataset_rte['train']['sentence1']
hypothesis_rte = dataset_rte['train']['sentence2']

In [ ]:
label_rte = dataset_rte['train']['gold_label']

In [ ]:
text_pos_id = np.arange(1, 1500, 3)

text_rte_pos = []
hypothesis_rte_pos = []
text_rte_neg = []
hypothesis_rte_neg = []
for i in text_pos_id:
  text_rte_pos.append(text_rte[i])
  hypothesis_rte_pos.append(hypothesis_rte[i-1])
  text_rte_neg.append(text_rte[i-1])
  hypothesis_rte_neg.append(hypothesis_rte[i])

### original data

In [ ]:
data_files = ['corpus/RTE/train.tsv', 'corpus/RTE/dev.tsv']
dataset_rte_origninal = load_dataset('csv', data_files=data_files, sep='\t')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2b6f8d73b37ce6e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_rte_origninal

DatasetDict({
    train: Dataset({
        features: ['index', 'sentence1', 'sentence2', 'label'],
        num_rows: 2767
    })
})

In [ ]:
text_rte_original = dataset_rte_origninal["train"]['sentence1']
hypothesis_rte_original = dataset_rte_origninal["train"]['sentence2']

In [ ]:
label_rte_original = dataset_rte_origninal["train"]["label"]

In [ ]:
hypothesis_rte_original_strip = []
for x in hypothesis_rte_original:
    hypothesis_rte_original_strip.append(x.strip())

In [ ]:
list_res_rte = {}
list_error_rte = []
for i, h in enumerate(hypothesis_rte_pos):
  h = h.strip()
  if h in hypothesis_rte_original_strip:
    index = hypothesis_rte_original_strip.index(h)
    label = label_rte_original[index]
    list_res_rte[i] = label
  else:
    list_error_rte.append((i,h))

In [ ]:
print(len(list_res_rte))
# list_res_rte 

500


In [ ]:
print(len(list_error_rte))

0


In [ ]:
list_final_rte=[]
for i in range(len(list_res_rte)):
    list_final_rte.append([text_rte_pos[i],hypothesis_rte_pos[i],list_res_rte[i]])
    list_final_rte.append([text_rte_neg[i],hypothesis_rte_pos[i], label_rte[i*3+0]])
    list_final_rte.append([text_rte_pos[i],hypothesis_rte_neg[i], label_rte[i*3+1]])
    list_final_rte.append([text_rte_neg[i],hypothesis_rte_neg[i], label_rte[i*3+2]])
 
column_name = ['text', 'hypothesis', 'label']
csv_name='./data/rte_completed.tsv'
xml_df = pd.DataFrame(list_final_rte, columns=column_name)
xml_df.to_csv(csv_name, sep='\t')